# Ensemble Learning

## Initial Imports

In [52]:
import warnings
warnings.filterwarnings('ignore')

In [53]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [54]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier

## Read the CSV and Perform Basic Data Cleaning

In [55]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

c= Counter(df.loan_status)
print(c.most_common())

df.describe(include='all')

[('low_risk', 68470), ('high_risk', 347)]


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817,6.881700e+04,68817,68817,68817,68817,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817,68817
unique,NaN,NaN,NaN,4,NaN,3,3,2,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
top,NaN,NaN,NaN,MORTGAGE,NaN,Not Verified,Jan-2019,low_risk,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N
freq,NaN,NaN,NaN,36219,NaN,32895,31041,68470,68817,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68817,68817
mean,16677.594562,0.127718,480.652863,NaN,8.821371e+04,NaN,NaN,NaN,NaN,21.778153,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,NaN,NaN
std,10277.348590,0.048130,288.062432,NaN,1.155800e+05,NaN,NaN,NaN,NaN,20.199244,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,NaN,NaN
min,1000.000000,0.060000,30.890000,NaN,4.000000e+01,NaN,NaN,NaN,NaN,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,NaN,NaN
25%,9000.000000,0.088100,265.730000,NaN,5.000000e+04,NaN,NaN,NaN,NaN,13.890000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,NaN,NaN
50%,15000.000000,0.118000,404.560000,NaN,7.300000e+04,NaN,NaN,NaN,NaN,19.760000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,NaN,NaN
75%,24000.000000,0.155700,648.100000,NaN,1.040000e+05,NaN,NaN,NaN,NaN,26.660000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,NaN,NaN


## Split the Data into Training and Testing

In [56]:
# Create our features
cols = df.columns.to_list()
cols.remove('loan_status')

X = df[cols]

# Create our target
y = df['loan_status']

In [57]:
X.describe(include = 'all')

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817,6.881700e+04,68817,68817,68817,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817,68817
unique,NaN,NaN,NaN,4,NaN,3,3,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
top,NaN,NaN,NaN,MORTGAGE,NaN,Not Verified,Jan-2019,n,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N
freq,NaN,NaN,NaN,36219,NaN,32895,31041,68817,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68817,68817
mean,16677.594562,0.127718,480.652863,NaN,8.821371e+04,NaN,NaN,NaN,21.778153,0.217766,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,NaN,NaN
std,10277.348590,0.048130,288.062432,NaN,1.155800e+05,NaN,NaN,NaN,20.199244,0.718367,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,NaN,NaN
min,1000.000000,0.060000,30.890000,NaN,4.000000e+01,NaN,NaN,NaN,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,NaN,NaN
25%,9000.000000,0.088100,265.730000,NaN,5.000000e+04,NaN,NaN,NaN,13.890000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,NaN,NaN
50%,15000.000000,0.118000,404.560000,NaN,7.300000e+04,NaN,NaN,NaN,19.760000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,NaN,NaN
75%,24000.000000,0.155700,648.100000,NaN,1.040000e+05,NaN,NaN,NaN,26.660000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,NaN,NaN


In [58]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [59]:
X_bin_encoded = pd.get_dummies(X)

In [60]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X_bin_encoded, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

In [61]:
y_train.value_counts()

low_risk     51352
high_risk      260
Name: loan_status, dtype: int64

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [62]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [63]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
scaler.fit(X_train)

StandardScaler()

In [64]:
# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [68]:
# Resample the training data with the BalancedRandomForestClassifier
# Import Balanced Random Forest Classifier from Imbalanced Learn Ensemble lib 
from imblearn.ensemble import BalancedRandomForestClassifier

# Create and fit the Balanced-RF Classifier
brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf_model.fit(X_train_scaled, y_train)

# Perform the prediction
y_pred_rf = brf_model.predict(X_test_scaled)

AttributeError: can't set attribute

In [66]:
# Calculated the balanced accuracy score


In [67]:
# Display the confusion matrix


In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

In [ ]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE

### Easy Ensemble Classifier

In [ ]:
# Train the Classifier
# YOUR CODE HERE

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.